TelegramBot

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import telebot
import pandas as pd
import os
import re  # Importa o módulo re para expressões regulares

from telebot import apihelper

apihelper.ENABLE_MIDDLEWARE = True

Substitua 'YOUR_TOKEN' pelo token do seu bot

In [ ]:
bot = telebot.TeleBot("...........")

Cria uma lista para armazenar os links

In [ ]:
links = []

Função para salvar links em uma planilha

In [ ]:
def save_links_to_excel():
    df = pd.DataFrame(links, columns=["Links"])
    df.to_excel("links.xlsx", index=False)

Função para salvar vídeos em uma pasta

In [ ]:
def save_video(file_id):
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    # Cria a pasta se não existir
    if not os.path.exists('videos'):
        os.makedirs('videos')
    # Salva o vídeo na pasta
    with open(f'videos/{file_info.file_path.split("/")[-1]}', 'wb') as new_file:
        new_file.write(downloaded_file)

Manipulador de mensagens

In [ ]:
@bot.middleware_handler(update_types=['message'])
def modify_message(bot_instance, message):
    # modifying the message before it reaches any other handler 
    #message.another_text = message.text + ':changed'
    print(f"Content type = {message.content_type}")

@bot.message_handler(content_types=['text', 'video'])
def handle_message(message):
    if message.content_type == 'text':
        # Verifica se o texto é um link que começa com 'https://'
        if re.match(r'^https://', message.text):
            links.append(message.text)
            save_links_to_excel()
            bot.reply_to(message, "Link salvo com sucesso!")
        else:
            bot.reply_to(
                message, "Por favor, envie um link válido que comece com 'https://'.")
    elif message.content_type == 'video':
        save_video(message.video.file_id)
        bot.reply_to(message, "Vídeo salvo com sucesso!")

Inicia o bot

In [ ]:
bot.infinity_polling()